# From script to project


## Typical steps

1. Organize your script into a package and modules
2. Add install scripts/instructions
3. Add documentation and make it available online
4. Add tests
5. Deploy your application/library

The final version of this lecture are available here:

Source code:
https://github.com/UiO-IN3110/monty_hall_game

Online documentation: https://uio-monty-hall-pre.readthedocs.io

Automatic testing:
https://github.com/UiO-IN3110/monty_hall_game/actions


## Our test case


We will use the `Monty Hall Game` implementation from last week as an example.


<img src="images/monty_hall_game.png" style="width: 400px;"/>


This `project` currently consists of the game file itself and some html templates:


```bash
monty_hall_game/
    game_server.py     # Start web server
    templates/*.html   # Templates for web-server
```


**Goal**:
Make the game available as a project with:

- a _command line_ interface
- a _web_ interface
- online and offline documentation
- tests
- error handling.


## Step 1. Organize your script into modules and functions.


We would like to offer a command line interface and a web interface. To achieve this,
we separate the game logic into a separate package (which is simply a directory with Python modules).

```bash
myproject/
    monty_hall_game/               # Game package
        __init__.py                    #    Init file module
        monty_hall_game.py             #    Core game module
```

The core game module contains the class `MontyHallGame`, which implements the core functionality of the game. Here is a the user interface for the game package:


Let's look at the user interface of the package.

Setting up the game:


In [1]:
!tree monty-hall-game0

monty-hall-game0
├── game_server.py
└── templates
    ├── final.html
    ├── reselect1.html
    ├── reselect.html
    ├── select2.html
    └── select.html

1 directory, 6 files


In [2]:
!tree -I __pycache__ monty-hall-game1

monty-hall-game1
├── bin
│   ├── play_monty_hall_cli.py
│   └── play_monty_hall_web.py
├── monty_hall_game
│   ├── __init__.py
│   └── monty_hall_game.py
└── templates
    ├── final.html
    ├── reselect1.html
    ├── reselect.html
    ├── select2.html
    └── select.html

3 directories, 9 files


In [3]:
import os
import sys

sys.path.insert(0, os.path.join(os.getcwd(), "monty-hall-game1"))

In [4]:
import monty_hall_game

game = monty_hall_game.MontyHallGame()

We can now play one round:


In [5]:
game.select_door(door=1)

In [6]:
game.let_host_open_door()

3

In [7]:
game.select_door(1)

In [8]:
game.open_door()  # True == win, False == lose

False

Printing the game statistics:


In [9]:
print(game.statistics())

Changed and won: 0 out of 0
Not changed and won: 0 out of 1


With this package, we can build scripts that expose the game to the user via the command line and the web-interface.
We implement these in the folder `bin` (for binary files. We use this name of convention reasons, even though our files are not really bindary files).


Our new directory structure is:


```bash
myproject/
    bin/                           # Scripts
        play_monty_hall_cli.py         #    Start game with command line interface
        play_monty_hall_web.py         #    Start game with web-server
    monty_hall_game/               # Game package
        __init__.py                    #    Init file module
        monty_hall_game.py             #    Core game module
    templates/*.html               # Templates for web-server
```


Let's look at the code in more detail (see files in `monty-hall-game1` folder)


## Step 2. Add installation files and instructions


### Record dependencies with requirements.txt


Our project has some dependencies to run it, such as `flask`,
and other dependencies to "develop" it such as pytest and (later) sphinx:


In [10]:
!cat monty-hall-game4/requirements.txt

flask


In [11]:
!cat monty-hall-game4/dev-requirements.txt

pytest
sphinx


The dependencies can be automatically installed with


In [12]:
%pip install -r monty-hall-game4/requirements.txt -r monty-hall-game4/dev-requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Setuptools

Further, we can create a setup file to simplify the installation of our game.
First thing we need is a `pyproject.toml` file:


In [13]:
%pycat monty-hall-game4/pyproject.toml

[build-system]
requires = [
    "setuptools",
]
build-backend = "setuptools.build_meta"

[project]
version = "0.1.0"
requires-python = ">=3.8"
license = {text = "MIT License"}

name = "monty_hall_game"
description = "..."
readme = "README.md"
dynamic = ["dependencies", "optional-dependencies"]


[tool.setuptools.dynamic]
dependencies = {file = ["requirements.txt",]}
optional-dependencies.dev = { file = ["dev-requirements.txt"] }

[project.scripts]
monty-hall-cli = "monty_hall_game.cli:main"
monty-hall-web = "monty_hall_game.web:main"


We note that we here have used a few new features compared to what we covered in [Structuring Python code](../../python/packages_and_testing.ipynb).

- `dynamic`: Some fields can be modified by other packages, scripts etc. when installing. One example is the list of dependencies, that we now let `setuptools` fetch from `requirements.txt`. See [Dynamic metadata](https://setuptools.pypa.io/en/latest/userguide/pyproject_config.html#dynamic-metadata) for more information on this. Similarly, we do the same thing for the `[dev]` extensions that are optional dependencies, only used to build the web client, but not run the `monty_hall_game`.
- `[project-scripts]`: creates an alias from calling the `monty_hall_game.cli.main()` function from the command line as `monty-hall-cli`. The syntax is `package.submodule:function`.


We can now install the `monty_hall_game` package with

```bash
python3 -m pip install .
```


Using the Python package manager `pip` has the advantage that we can uninstall the package again:

```bash
python3 -m pip uninstall monty_hall_game
```


In [14]:
!tree monty-hall-game4/

monty-hall-game4/
├── build
│   ├── bdist.linux-x86_64
│   └── lib
│       └── monty_hall_game
│           ├── cli.py
│           ├── game_exceptions.py
│           ├── __init__.py
│           ├── monty_hall_game.py
│           └── web.py
├── dev-requirements.txt
├── dist
│   ├── monty_hall_game-0.1.0-py3-none-any.whl
│   └── monty_hall_game-0.1.0.tar.gz
├── docs
│   ├── api
│   │   ├── index.rst
│   │   └── monty_hall_game.rst
│   ├── _build
│   │   ├── doctrees
│   │   │   ├── api
│   │   │   │   ├── index.doctree
│   │   │   │   └── monty_hall_game.doctree
│   │   │   ├── environment.pickle
│   │   │   └── index.doctree
│   │   └── html
│   │       ├── api
│   │       │   ├── index.html
│   │       │   └── monty_hall_game.html
│   │       ├── genindex.html
│   │       ├── index.html
│   │       ├── _modules
│   │       │   ├── index.html
│   │       │   └── monty_hall_game
│   │       │       ├── game_exceptions.html
│   │       │       └── monty_hall_game.html
│   │       ├── objec

### Installation instructions


Finally it is good practice to add a installation instructions to the README.md file:


In [15]:
from IPython.display import Markdown, display

with open("monty-hall-game4/README.md") as f:
    display(Markdown(f.read()))

# Monty Hall Game

This repository contains a simple implementation of the Monty Hall Game with
a command line and web interface.

## Installation

Install the game with

    pip install .

## Running the game

The command line interface is started with:

    monty-hall-cli

The web server is started with:

    monty-hall-web


### New files

```bash
monty_hall_game/
    README.md                   # Installation instructions
    requirements.txt            # List of project dependencies
    dev-requirements.txt        # List of development dependencies
    pyproject.toml              # SetupTools file
```


In [16]:
!pwd

/home/dokken/Documents/UiO-IN3110.github.io/lectures/production/from-script-to-project


In [17]:
%cd monty-hall-game4/
%pip install .

/home/dokken/Documents/UiO-IN3110.github.io/lectures/production/from-script-to-project/monty-hall-game4
Defaulting to user installation because normal site-packages is not writeable
Processing /home/dokken/Documents/UiO-IN3110.github.io/lectures/production/from-script-to-project/monty-hall-game4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for monty-hall-game: filename=monty_hall_game-0.1.0-py3-none-any.whl size=5185 sha256=176c9aaaa28fdffdbe02b26860a577bffc23094732217bf158dd39f3c8007c56
  Stored in directory: /home/dokken/.cache/pip/wheels/86/bc/e5/d84458101c63904d8a2ce3e8a90036d93b07da4125d14faf2d
Successfully built monty-hall-game
  Attempting uninstall: monty-hall-game
    Found existing installation: monty-hall-game 0.1.0
    Uninstalling monty-hall-game-0.1.0:
      Successfully uninstalled monty-hall-game-0.1.0
Note: you may need to

## Step 3. Add documentation


We should add docstrings to the module `monty_hall_game.py` file.

Note, that I write the documentation in a Sphinx Markup Style (see https://www.sphinx-doc.org/en/master/usage/extensions/napoleon.html) to obtain nicely rendered online documentation.

Let's look at the code in more detail (see files in `monty-hall-game2` folder)

Once done, we can access the docstrings as usual:


In [18]:
# update import path and re-import monty_hall_game
sys.path.insert(0, os.path.join(os.getcwd(), "monty-hall-game2"))
from importlib import reload

reload(monty_hall_game)
reload(monty_hall_game.monty_hall_game)
reload(monty_hall_game)

from monty_hall_game.monty_hall_game import MontyHallGame

In [19]:
MontyHallGame.let_host_open_door?

Signature: MontyHallGame.let_host_open_door(self)
Docstring: <no docstring>
File:      ~/Documents/UiO-IN3110.github.io/lectures/production/from-script-to-project/monty-hall-game1/monty_hall_game/monty_hall_game.py
Type:      function

## Python documentation with Sphinx

Sphinx is a powerful tool to create documentation for Python projects and provides more flexibiliy.

### Installation

```bash
python3 -m pip install sphinx
```

### How to get started

1. Use the quick start command to configure a base Sphinx documentation

   ```bash
   sphinx-quickstart
   ```

   Among other things, the quickstart guide will ask for the documentation folder. I typically choose `docs` for this.

2. Use
   ```bash
   mkdir -p docs/api
   sphinx-apidoc -o docs/api monty_hall_game
   ```
   to add documentation for each module.
3. Edit `docs/index.rst` to change the content of your main page.
4. Compile the documentation with:
   ```bash
   cd docs
   make html
   ```
   (make sure that the module is in sys.path or installed).
5. The documentation is available on `docs/_build/html/index.html`.


In [20]:
%cd docs
!make html

/home/dokken/Documents/UiO-IN3110.github.io/lectures/production/from-script-to-project/monty-hall-game4/docs
sphinx-build -b html -d _build/doctrees   . _build/html
Running Sphinx v4.5.0
loading pickled environment... done
building [mo]: targets for 0 po files that are out of date
building [html]: targets for 0 source files that are out of date
updating environment: 0 added, 1 changed, 0 removed
reading sources... [100%] api/monty_hall_game                                   
looking for now-outdated files... none found
pickling environment... done
checking consistency... done
preparing documents... done
writing output... [100%] index                                                  
generating indices... genindex py-modindex done
highlighting module code... [100%] monty_hall_game.monty_hall_game              
writing additional pages... search done
copying static files... done
copying extra files... done
dumping search index in English (code: en)... done
dumping object inventory... don

#### New files (autogenerated with `sphinx-quickstart` and `sphinx-apidoc`)

```bash
    docs/
        conf.py              # Sphinx configuration file
        index.rst            # Index page (in markdown format)
        make.bat             # Windows build file
        Makefile             # Linux/MacOS build file
    docs/api
        modules.rst          # Module page
        monty_hall_game.rst  # Module page
```

## Step 4. Add tests

We will use `pytest` as testing framework.
New files:

```bash
monty_hall_game/
    tests
        test_game_module.py
```

We can run the test suite with


Let's look at the code in more detail (see files in `monty-hall-game4` folder)


In [28]:
!cd monty-hall-game4 && python3 -m pytest tests

============================= test session starts ==============================
platform linux -- Python 3.8.10, pytest-7.1.3, pluggy-1.0.0
rootdir: /home/dokken/Documents/UiO-IN3110.github.io/lectures/production/from-script-to-project/monty-hall-game4
plugins: anyio-3.6.1, cov-3.0.0
collected 3 items                                                              

tests/test_game_module.py ...                                            [100%]

============================== 3 passed in 0.02s ===============================


## Continuous integration with GitHub Actions

[GitHub Actions docs](https://docs.github.com/actions)

### Quick guide to GitHub action

create `.github/workflows/test.yml` (any `name.yml` will do, you can have several) with steps:

1. checkout the repo
2. install Python
3. install your package and its dependencies
4. run the tests

```yaml
# This is a GitHub workflow defining a set of jobs with a set of steps.
# ref: https://docs.github.com/en/free-pro-team@latest/actions/reference/workflow-syntax-for-github-actions
#
name: Tests

on:
  pull_request:
  push:

jobs:
  test:
    runs-on: ubuntu-22.04

    strategy:
      matrix:
        python:
          - "3.9"
          - "3.10"

    steps:
      # checkout the repository
      - uses: actions/checkout@v3

      # setup Python
      - name: Install Python ${{ matrix.python }}
        uses: actions/setup-python@v4
        with:
          python-version: ${{ matrix.python }}

      # preserve pip cache to speed up installation
      - name: Cache pip
        uses: actions/cache@v3
        with:
          path: ~/.cache/pip
          # Look to see if there is a cache hit for the corresponding requirements file
          key: ${{ runner.os }}-pip-${{ hashFiles('*requirements.txt') }}
          restore-keys: |
            ${{ runner.os }}-pip-

      - name: Install Python dependencies
        run: |
          pip install --upgrade pip
          pip install --upgrade .[dev]
          pip freeze

      - name: Run tests
        run: |
          pytest -v --color=yes
```

### Publishing the documentation
Similarly, we can use Github actions to publish a webpage hosted by Github.
We will first make a stand-alone action that simply uploads the built web-page as an [artifact](https://docs.github.com/en/actions/using-workflows/storing-workflow-data-as-artifacts). This can be downloaded an inspected.

```yaml
ame: Build documentation

on:
    workflow_dispatch:
    workflow_call:
    pull_request:
        branches:
            - main 
jobs:
  build_docs:
    runs-on: ubuntu-22.04
    env:
        PYTHON_VERSION: "3.10"

    steps:
      # checkout the repository
      - uses: actions/checkout@v4

      # setup Python
      - name: Install Python ${{ env.PYTHON_VERSION }}
        uses: actions/setup-python@v4
        with:
          python-version: ${{ env.PYTHON_VERSION }}

      # preserve pip cache to speed up installation
      - name: Cache pip
        uses: actions/cache@v3
        with:
          path: ~/.cache/pip
          # Look to see if there is a cache hit for the corresponding requirements file
          key: ${{ runner.os }}-pip-${{ hashFiles('*requirements.txt') }}
          restore-keys: |
            ${{ runner.os }}-pip-
      - name: Install Python dependencies
        run: |
          python -m pip install --upgrade pip
          python -m pip install --upgrade .[dev]
          python -m pip freeze
      - name: Build documentation
        run: |
          cd docs
          make html
      - name: Upload documentation artifact
        uses: actions/upload-artifact@v3
        with:
            name: documentation
            path: ./docs/_build/html
            if-no-files-found: error

```

By adding `workflow_call` to the settings that decide when this is executed, we allow this workflow to be run from inside another workflow.
We can then create a Github publishing workflow, that only runs on the main branch, that pushes the web page to Github
```yaml
name: Github Pages

on:
  push:
    branches: [main]

# Sets permissions of the GITHUB_TOKEN to allow deployment to GitHub Pages
permissions:
  contents: read
  pages: write
  id-token: write

# Allow one concurrent deployment
concurrency:
  group: "pages"
  cancel-in-progress: true

jobs:  

  build-docs:
    uses: ./.github/workflows/build_docs.yml

  deploy:
    needs: [build-docs]

    environment:
      name: github-pages
      url: ${{ steps.deployment.outputs.page_url }}

    runs-on: ubuntu-latest
    steps:
      - name: Download docs artifact
        # docs artifact is uploaded by build-docs job
        uses: actions/download-artifact@v3
        with:
          name: documentation
          path: "./public"


      - name: Upload artifact
        uses: actions/upload-pages-artifact@v2
        with:
          path: "./public"

      - name: Checkout
        uses: actions/checkout@v4

      - name: Setup Pages
        uses: actions/configure-pages@v3

      - name: Deploy to GitHub Pages
        id: deployment
        uses: actions/deploy-pages@v2

```


Check:

https://github.com/UiO-IN3110/monty_hall_game/actions

Check the build status page to see if your build passes or fails according to the return status of the build command by visiting your GitHub repository and selecting `actions`.


## Current directory structure

```bash
monyty_hall_game/
    README.md                      # Installation instructions
    requirements.txt               # Dependencies
    setup.py                       # Setuptools
    monty_hall_game/               # Main module
        __init__.py
        web.py
        cli.py
        game_exceptions.py
        monty_hall_game.py
        templates/*.html
        static/*.{png,jpg}
    docs/                          # Sphinx documentation (mostly autogenerated)
        conf.py
        index.rst
        Makefile
        modules.rst
        monty_hall_game.rst
    tests                          # tests in pytest format
        test_game_module.py
    .github/workflows/test.yml     # continuous integration
    .gitignore
```


## Deploy your Python library to PyPI

Wouldn't it be nice if your users could just type in:

```bash
python3 -m pip install monty_hall_game
```

This is possible by uploading the package to the Python Package Index (PyPI).

1. Create a source and wheel distribution with the following command:

```bash
python -m pip install build
python -m build .
```

2. Upload the distribution to pypi (will ask for the pypi credentials) with:

```bash
twine upload -r test dist/*
```

Result see https://test.pypi.org/project/uio-monty-hall-game/


In [29]:
%cd monty-hall-game4
!rm -rf dist
!python3 -m pip install build
!python3 -m build .

/home/dokken/Documents/UiO-IN3110.github.io/lectures/production/from-script-to-project/monty-hall-game4
Defaulting to user installation because normal site-packages is not writeable
* Creating venv isolated environment...
* Installing packages in isolated environment... (setuptools)
* Getting build dependencies for sdist...
running egg_info
writing monty_hall_game.egg-info/PKG-INFO
writing dependency_links to monty_hall_game.egg-info/dependency_links.txt
writing entry points to monty_hall_game.egg-info/entry_points.txt
writing requirements to monty_hall_game.egg-info/requires.txt
writing top-level names to monty_hall_game.egg-info/top_level.txt
reading manifest file 'monty_hall_game.egg-info/SOURCES.txt'
writing manifest file 'monty_hall_game.egg-info/SOURCES.txt'
* Building sdist...
running sdist
running egg_info
writing monty_hall_game.egg-info/PKG-INFO
writing dependency_links to monty_hall_game.egg-info/dependency_links.txt
writing entry points to monty_hall_game.egg-info/entry_poi

In [30]:
ls dist

monty_hall_game-0.1.0-py3-none-any.whl  monty_hall_game-0.1.0.tar.gz


In [32]:
!twine upload -r test dist/*

Uploading distributions to https://test.pypi.org/legacy/
Uploading uio_monty_hall_game-2022.11.17-py3-none-any.whl
100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 kB • 00:00 • 278.3 kB/s
Uploading uio-monty-hall-game-2022.11.17.tar.gz
100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 kB • 00:00 • ?

View at:
https://test.pypi.org/project/uio-monty-hall-game/2022.11.17/


## Final directory layout


In [31]:
!tree -I __pycache__ .

.
├── build
│   ├── bdist.linux-x86_64
│   └── lib
│       └── monty_hall_game
│           ├── cli.py
│           ├── game_exceptions.py
│           ├── __init__.py
│           ├── monty_hall_game.py
│           └── web.py
├── dev-requirements.txt
├── dist
│   ├── monty_hall_game-0.1.0-py3-none-any.whl
│   └── monty_hall_game-0.1.0.tar.gz
├── docs
│   ├── api
│   │   ├── index.rst
│   │   └── monty_hall_game.rst
│   ├── _build
│   │   ├── doctrees
│   │   │   ├── api
│   │   │   │   ├── index.doctree
│   │   │   │   └── monty_hall_game.doctree
│   │   │   ├── environment.pickle
│   │   │   └── index.doctree
│   │   └── html
│   │       ├── api
│   │       │   ├── index.html
│   │       │   └── monty_hall_game.html
│   │       ├── genindex.html
│   │       ├── index.html
│   │       ├── _modules
│   │       │   ├── index.html
│   │       │   └── monty_hall_game
│   │       │       ├── game_exceptions.html
│   │       │       └── monty_hall_game.html
│   │       ├── objects.inv
│   │    

```bash
monyty_hall_game/
    README.md                      # Installation instructions
    requirements.txt               # Dependencies
    setup.py                       # Setuptools
    monty_hall_game/               # Our package
        __init__.py
        game_exceptions.py
        monty_hall_game.py

    bin/                           # Scripts
        play_monty_hall_cli.py
        play_monty_hall_web.py
        templates/*.html
    docs/                          # Sphinx documentation (mostly autogenerated)
        conf.py
        index.rst
        Makefile
        modules.rst
        monty_hall_game.rst
    tests                          # tests in pytest format
        test_game_module.py
    .github/workflows/test.yml     # continuous integration
    .gitignore
```


## Summary of today's topics

- **Installation**
  - requirements.txt
  - Package + pyproject.toml
  - README.md
- **Testing**
  - Continuous integration (with GitHub)
- **Documentation**
  - Sphinx
  - ReadTheDocs
- **Deployment / Publishing**
  - Deployment of Python packages with PyPI
